In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_train

,Id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


In [3]:
import numpy as np

print(np.__path__)

train0 = np.load('train/train/0.npy')
train0

['/home/ubuntu/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/numpy']


array([[nan, nan,  1., ..., nan, nan, nan],
       [nan, nan,  2., ..., 19., nan, nan],
       [nan, nan,  3., ..., 22., nan, nan],
       ...,
       [nan, nan, 52., ..., 26., nan, nan],
       [nan, nan, 53., ..., 22., nan, nan],
       [nan, nan, 54., ..., 18., nan, nan]])

In [4]:
def load_dataframe(id):
    train_data = np.load("train/train/{}.npy".format(id))
    return pd.DataFrame(data=train_data)

In [5]:
def get_missing_data(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [6]:
def clear_missing_data(df, missing_indices):
    df = df.drop(missing_indices,1)
    # for col in df_temp.columns:
    #    df = df.drop(df.loc[df[col].isnull()].index)
    count = df.isnull().sum().max() #just checking that there's no missing data missing...
    if count > 0:
        print(count, "NaN")
        return pd.DataFrame()
    return df

In [7]:
def impute_missing_data_with_average(df, drop_indices, average):
    df = df.drop(drop_indices, 1)
    for col in df.columns:
        df[col] = df[col].fillna(average[col])
    return df

In [8]:
def impute_missing_data_with_zero(df, drop_indices):
    df = df.drop(drop_indices, 1)
    for col in df.columns:
        df[col] = df[col].fillna(0)
    return df

In [9]:
df8 = load_dataframe(8)
missing_df8 = get_missing_data(df8)
missing_indices = missing_df8[missing_df8['Total'] > 0].index.append(pd.Index([2,11,33,35]))
cleaned_df8 = clear_missing_data(df8, missing_indices)

#custom to add some column back
adding_cols = [22]
to_be_removed = []
for idx, i in enumerate(missing_indices):
    if i in adding_cols:
        to_be_removed.append(idx)
missing_indices = missing_indices.delete(to_be_removed)

missing_indices.sort_values(ascending=True)

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39],
           dtype='int64')

In [10]:
y = df_train['label']
y

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        1
9        0
10       1
11       0
12       0
13       0
14       1
15       0
16       0
17       1
18       0
19       0
20       0
21       1
22       0
23       0
24       0
25       0
26       0
27       1
28       0
29       0
        ..
30306    0
30307    0
30308    0
30309    0
30310    0
30311    0
30312    0
30313    0
30314    0
30315    0
30316    0
30317    0
30318    0
30319    0
30320    0
30321    1
30322    0
30323    0
30324    0
30325    0
30326    0
30327    0
30328    0
30329    0
30330    0
30331    0
30332    0
30333    1
30334    0
30335    1
Name: label, Length: 30336, dtype: int64

In [11]:
# Calculate average for each feature column and store it as npy
# import sys

# average = np.empty(40)
# accumulated_count = [0] * df_train.shape[0]
# whole_col_same_value = [35]
# for id in df_train['Id']:
#     if id % 10 == 0:
#         sys.stdout.write("\rprocessing %i" % id)
# #         sys.stdout.write("\r {} {}".format(id, average[1]))
#         sys.stdout.flush()
#     df = load_dataframe(id)
#     for i in range(40):
#         if i in whole_col_same_value:
#             value = df.values[0][i]
#             if np.isnan(value):
#                 continue
#             average[i] = (average[i] * accumulated_count[i] + df.values[0][i]) / (accumulated_count[i] + 1)
#             accumulated_count[i] += 1
#         else:
#             values = df.values[:,i]
#             is_not_nan = ~np.isnan(values)
#             values = values[is_not_nan]
#             if len(values) == 0:
#                 continue
#             count = len(values)
#             average[i] = (average[i] * accumulated_count[i] + np.sum(values)) / (accumulated_count[i] + count)
#             accumulated_count[i] += count

# np.save("average", average)
# average

In [12]:
average = np.load("average.npy")
average

array([  0.49555807,   2.04680609,  27.15729269, 122.83364621,
         2.28476449,   2.57970877,  10.49785743,   0.50444193,
         4.14589169,   1.46722637,   2.13536761,  62.34371757,
       135.96647116,   0.55852966,   7.79536143,  84.76420289,
       289.8111412 ,  81.18010694,   7.37931228,   3.561608  ,
        92.04059017,  24.04156655,  41.11530986, 105.79398754,
        62.82663102,  33.57663965,  -0.66819929, 197.20136227,
       106.01175318,  97.22616487, 291.42421967,  30.76740307,
        11.60964474,   0.56573611,   8.7685371 , -50.54944157,
        24.08684309,  18.74176613,  41.006491  ,  36.99253031])

In [13]:
def get_cleaned_dataframes(exclude_indices):
    dataframes = []
    empty_dfi = []

    for id in df_train['Id']:
        if id % 10 == 0:
            sys.stdout.write("\rprocessing %i" % id)
        df = load_dataframe(id)

        #remove any columns with nan
    #     dfi = clear_missing_data(df, exclude_indices)
    #     if dfi.empty:
    #         empty_dfi.append(id)
    #         avg_data = {}
    #         for missing_id in np.delete(np.array(range(40)), exclude_indices):
    #             avg_data[missing_id] = np.full(df.shape[0],average[missing_id])
    #         dfi = pd.DataFrame(data=avg_data)

        #remove only if given and impute some values
        dfi = impute_missing_data_with_average(df, exclude_indices, average)

        #remove only if given and set NaN as 0
    #     dfi = impute_missing_data_with_zero(df, exclude_indices)

        dfi['Id'] = id

        dataframes.append(dfi)
    print("\nempty dfi", empty_dfi)
    return dataframes

In [14]:
# import sys
# !conda install --yes --prefix {sys.prefix} -c conda-forge tsfresh

In [15]:
# import sys
# !pip install --prefix {sys.prefix} fastparquet

In [16]:
# Save extracted features for each feature column
# from tsfresh import extract_features

# all_indices = np.arange(40)
# for i in all_indices:
#     print("feature", i)
#     removing_indices = np.delete(all_indices, i)
#     df_train_all = pd.concat(get_cleaned_dataframes(removing_indices), ignore_index=True)
#     extracted_features = extract_features(df_train_all, column_id="Id", show_warnings=False)
#     extracted_features.to_parquet("feature" + i.astype(str) + ".gzip", compression="gzip")

In [17]:
# Save selected features
# from tsfresh import select_features
# from tsfresh.utilities.dataframe_functions import impute

# all_indices = np.arange(40)
# for i in all_indices:
#     print("feature", i)
#     removing_indices = np.delete(all_indices, i)
#     extracted_features = pd.read_parquet("feature" + i.astype(str) + ".gzip")
#     impute(extracted_features)
#     features_filtered = select_features(extracted_features, y)
#     features_filtered.to_parquet("filtered_feature" + i.astype(str) + ".gzip", compression="gzip")


In [18]:
features_filtered = pd.read_parquet("filtered_feature22.gzip")

In [19]:
from sklearn import preprocessing

XTrain = preprocessing.scale(features_filtered)
XTrain

/home/ubuntu/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


array([[ 1.46041078e+00, -8.52021118e-02,  1.45430573e+00, ...,
         6.50211330e-03, -2.01160222e+01,  6.11584062e+00],
       [-9.95134230e-02, -8.52021118e-02, -1.15997967e-01, ...,
         6.50211330e-03, -1.05370164e-02, -1.99155150e-02],
       [ 4.41476686e-01, -8.52021118e-02,  4.36878686e-01, ...,
         6.50211330e-03,  3.44329245e-01, -9.36048027e-02],
       ...,
       [ 7.40351496e-01, -8.52021118e-02,  7.63837156e-01, ...,
         6.50211330e-03, -1.05370164e-02,  1.32641416e-01],
       [ 5.07866133e-01, -8.52021118e-02,  5.05965659e-01, ...,
         6.50211330e-03, -1.05370164e-02, -1.99155150e-02],
       [ 3.45261240e-01, -8.52021118e-02,  3.35987582e-01, ...,
         6.50211330e-03, -1.05370164e-02, -1.99155150e-02]])

In [20]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import cross_val_score

# gb_clf = GradientBoostingClassifier()
# scores = cross_val_score(gb_clf, XTrain, y, cv=5)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# scores

In [21]:
# gb_clf.fit(XTrain, y)

In [35]:
import sys

def load_dataframe_test(id):
    train_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=train_data)

df_test = pd.read_csv('sample_solution.csv')
def get_cleaned_test_dataframes(exclude_indices):
    dataframestest = []
    empty_dfi = []
    for id in df_test['Id']:
        if id % 10 == 0:
            sys.stdout.write("\rprocessing %i" % id)
        df = load_dataframe_test(id)

        #remove any columns with nan
    #     dfi = clear_missing_data(df, exclude_indices)
    #     if dfi.empty:
    # #         print("{}:".format(id))
    #         empty_dfi.append(id)
    #         continue
        #remove only if given and impute some values
        dfi = impute_missing_data_with_average(df, exclude_indices, average)

        #remove only if given and set NaN as 0
    #     dfi = impute_missing_data_with_zero(df, exclude_indices)

        dfi['Id'] = id
        dataframestest.append(dfi)
    print("\nempty dfi", empty_dfi)
    return dataframestest

# df_test_all = pd.concat(get_cleaned_test_dataframes(missing_indices), ignore_index=True)

In [23]:
#generate extracted features for each feature column for test data
# from tsfresh import extract_features

# all_indices = np.arange(40)
# for i in all_indices:
#     print("test feature", i)
#     removing_indices = np.delete(all_indices, i)
#     df_test_all = pd.concat(get_cleaned_test_dataframes(removing_indices), ignore_index=True)
#     extracted_features_test = extract_features(df_test_all, column_id="Id", show_warnings=False)
#     extracted_features_test.to_parquet("test_feature" + i.astype(str) + ".gzip", compression="gzip")

In [24]:
#generate selected extracted features for each feature column for test data
# all_indices = np.arange(40)
# for i in all_indices:
#     print("test feature", i)
#     extracted_features_test = pd.read_parquet("test_feature" + i.astype(str) + ".gzip")
#     features_filtered = pd.read_parquet("filtered_feature" + i.astype(str) + ".gzip")
#     test_features_filtered = extracted_features_test[features_filtered.columns]
#     test_features_filtered.to_parquet("filtered_test_feature" + i.astype(str) + ".gzip", compression="gzip")

In [25]:
# extracted_features_test = pd.read_parquet("test_feature22.gzip")
XTest = pd.read_parquet("filtered_test_feature22.gzip")
XTest

variable,"22__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_20","22__agg_linear_trend__f_agg_""min""__chunk_len_50__attr_""stderr""","22__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_20","22__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_20","22__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_20","22__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_20","22__agg_linear_trend__f_agg_""var""__chunk_len_50__attr_""stderr""","22__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_20","22__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_3__w_20","22__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""stderr""",...,"22__fft_coefficient__coeff_32__attr_""real""","22__fft_coefficient__coeff_0__attr_""real""","22__fft_coefficient__coeff_0__attr_""abs""",22__sum_values,"22__fft_coefficient__coeff_83__attr_""real""","22__fft_coefficient__coeff_58__attr_""real""",22__skewness,"22__fft_coefficient__coeff_75__attr_""imag""","22__fft_coefficient__coeff_22__attr_""real""",22__c3__lag_2
id,,,,,,,,,,,,,,,,,,,,,
0,155.824560,0.0,150.035247,143.795900,137.145109,130.126385,0.000000,122.787695,115.180917,0.000000,...,NaN,2245.996112,2245.996112,2245.996112,NaN,NaN,5.462388,NaN,-1.915927e+01,71449.494067
1,161.621870,NaN,155.652466,149.265173,142.500389,135.403027,NaN,128.022028,120.409793,NaN,...,NaN,1644.612394,1644.612394,1644.612394,NaN,NaN,0.000000,NaN,NaN,69504.144586
2,158.040090,NaN,152.070686,145.683393,138.918609,131.821248,NaN,124.440248,116.828014,NaN,...,NaN,1973.534873,1973.534873,1973.534873,NaN,NaN,0.000000,NaN,0.000000e+00,69504.144586
3,150.030896,0.0,144.284059,138.100960,131.519579,124.582628,0.000000,117.337100,109.833740,0.000000,...,NaN,2343.572662,2343.572662,2343.572662,NaN,NaN,0.000000,NaN,7.080147e-14,69504.144586
4,161.665271,NaN,155.918434,149.735335,143.153954,136.217003,NaN,128.971475,121.468114,NaN,...,NaN,1932.419563,1932.419563,1932.419563,NaN,NaN,0.000000,NaN,-3.730349e-13,69504.144586
5,129.746974,NaN,123.777571,117.390277,110.625493,103.528132,NaN,96.147133,88.534898,NaN,...,NaN,740.075577,740.075577,740.075577,NaN,NaN,0.000000,NaN,NaN,69504.144586
6,135.380389,NaN,129.633553,123.450454,116.869072,109.932122,NaN,102.686594,95.183233,NaN,...,NaN,781.190887,781.190887,781.190887,NaN,NaN,0.000000,NaN,NaN,69504.144586
7,142.418289,NaN,136.751381,130.669341,124.208908,117.411324,NaN,110.321904,102.989538,NaN,...,NaN,929.536817,929.536817,929.536817,NaN,NaN,-4.795832,NaN,NaN,65202.708748
8,129.746974,NaN,123.777571,117.390277,110.625493,103.528132,NaN,96.147133,88.534898,NaN,...,NaN,740.075577,740.075577,740.075577,NaN,NaN,0.000000,NaN,NaN,69504.144586


In [26]:
XTest = pd.read_parquet("filtered_test_feature22.gzip")
XTest = XTest.replace([np.inf], np.finfo('float32').max).replace([np.inf, -np.inf], np.finfo('float32').min).fillna(0)
print(np.all(np.isfinite(XTest)))
# XTest

True


In [27]:
XTest = preprocessing.scale(XTest)
XTest

array([[ 3.80288410e-01, -8.49853051e-02,  3.75400644e-01, ...,
        -6.34429444e-03, -3.78362827e+00,  9.15105580e-01],
       [ 5.74210928e-01, -8.49853051e-02,  5.86548695e-01, ...,
        -6.34429444e-03, -5.08582124e-04,  3.74900391e-02],
       [ 4.54398845e-01, -8.49853051e-02,  4.51911648e-01, ...,
        -6.34429444e-03, -5.08582124e-04,  3.74900391e-02],
       ...,
       [ 5.18536830e-01, -8.49853051e-02,  5.32351931e-01, ...,
        -6.34429444e-03, -5.08582124e-04,  3.74900391e-02],
       [ 6.49846881e-01, -8.49853051e-02,  6.79909649e-01, ...,
        -6.34429444e-03, -5.08582124e-04,  3.74900391e-02],
       [ 5.66023382e-01, -8.49853051e-02,  5.77348062e-01, ...,
        -6.34429444e-03, -5.08582124e-04,  3.74900391e-02]])

In [28]:
# import sys
# !pip install --prefix {sys.prefix} xgboost

In [74]:
# Tuning XGBoost
from xgboost import XGBClassifier
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import train_test_split

train, test = train_test_split(pd.DataFrame(data=XTrain), test_size=0.2)
ytrain = y.drop(test.index) 
ytest = y.drop(train.index)
xgb_clf = XGBClassifier(max_depth=3, 
                        learning_rate=0.1, 
                        n_estimators=100, 
                        verbosity=1, 
                        objective='binary:logistic', 
                        booster='gbtree', 
                        tree_method='auto', 
                        n_jobs=1, 
                        gamma=0, 
                        min_child_weight=1, 
                        max_delta_step=0, 
                        subsample=1, 
                        colsample_bytree=1, 
                        colsample_bylevel=1, 
                        colsample_bynode=1, 
                        reg_alpha=0, 
                        reg_lambda=1, 
                        scale_pos_weight=1, 
                        base_score=0.5,
                        random_state=0)
#depth 3 0.7832
#depth 10 0.81044

xgb_clf.fit(train, ytrain)
probs = xgb_clf.predict_proba(test)
loss = brier_score_loss(ytest.to_numpy(), probs[:,0])
print("Loss: ", loss)

('Loss: ', 0.7874340378136128)


In [30]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
xgb_clf.fit(XTrain, y)
probs = xgb_clf.predict_proba(XTest)
probs

array([[0.9792148 , 0.0207852 ],
       [0.97279674, 0.02720326],
       [0.97894156, 0.02105846],
       ...,
       [0.98334086, 0.01665914],
       [0.98155504, 0.01844494],
       [0.98155504, 0.01844494]], dtype=float32)

In [ ]:
# probs = gb_clf.predict_proba(XTest)
# probs

In [33]:
res = probs[:, 0]

In [36]:
df_test['Predicted'] = res
df_test

,Id,Predicted
0,0,0.979215
1,1,0.972797
2,2,0.978942
3,3,0.971067
4,4,0.981375
5,5,0.928702
6,6,0.947418
7,7,0.854871
8,8,0.928702
9,9,0.978384


In [37]:
df_test.to_csv('test.csv', index=False)